In [21]:
import pandas as pd

df = pd.read_csv('data/gap/gap-development.tsv', sep='\t')

In [2]:
df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)

embeds = bert_tokenizer.encode_plus(df['Text'][0], is_split_into_words=False, return_tensors="pt")

print(embeds)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101, 11929, 11341,  2821,   118,   118,  1307,  1103,  2021,  2575,
          6124,  1104,  3274,   117,  8153,   119, 12786,  8223,  1174,  1118,
          3274,  1107,  1103,  1509,  2004,  1104,  1326,   122,   117,  1170,
          1119,  7362,  1114,  8067,   117,  1105,  1110,  1136,  1562,  1254,
           119, 19704,  1819,  1307, 21173, 14190,   117, 16473,   112,   188,
          1910,  1105,  1145,   170,  1214,  1429, 11602,  1107,  3274,   112,
           188,  1705,   119, 12786,  8223,  1174,  1123,  6508,  1378,  3274,
           112,   188,  5566,  1170,  1119,  2010,   112,   189,  1138,  2673,
          1114,  1123,  1133,  1224, 11326,  1142,  1108,  1496,  1106,  1140,
          9256, 24121,  1116,  1228,  1123,  1910, 16473,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0,

In [4]:
input_ids = embeds['input_ids']
print(input_ids.shape)
print(bert_tokenizer.convert_ids_to_tokens(input_ids.squeeze()))

torch.Size([1, 99])
['[CLS]', 'Zoe', 'Tel', '##ford', '-', '-', 'played', 'the', 'police', 'officer', 'girlfriend', 'of', 'Simon', ',', 'Maggie', '.', 'Du', '##mp', '##ed', 'by', 'Simon', 'in', 'the', 'final', 'episode', 'of', 'series', '1', ',', 'after', 'he', 'slept', 'with', 'Jenny', ',', 'and', 'is', 'not', 'seen', 'again', '.', 'Phoebe', 'Thomas', 'played', 'Cheryl', 'Cassidy', ',', 'Pauline', "'", 's', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'Simon', "'", 's', 'class', '.', 'Du', '##mp', '##ed', 'her', 'boyfriend', 'following', 'Simon', "'", 's', 'advice', 'after', 'he', 'wouldn', "'", 't', 'have', 'sex', 'with', 'her', 'but', 'later', 'realised', 'this', 'was', 'due', 'to', 'him', 'catching', 'crab', '##s', 'off', 'her', 'friend', 'Pauline', '.', '[SEP]']


In [5]:
def get_word_embedding(encoded, output, idx):
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    print(token_ids_word)
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0)

encoded = bert_tokenizer.encode_plus(df['Text'][0], is_split_into_words=False, return_tensors="pt")
encoded_dict = {**encoded}
input_ids = encoded_dict['input_ids']
token_type_ids = encoded_dict['token_type_ids']
attention_mask = encoded_dict['attention_mask']

n = input_ids.shape[1]

passes = [(i * 256, i * 256 + 512) for i in range(int(n / 256) - 1)]
if len(passes) == 0:
    # Documents < 512 tokens
    with torch.no_grad():
        outputs = bert_model(**encoded)

    # Get all hidden states
    states = outputs.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i].squeeze() for i in [-4, -3, -2, -1]])
    query_t = torch.mean(output, dim=0)
    print(get_word_embedding(encoded, query_t, 90))

(array([97], dtype=int64),)
tensor([ 2.4865e-02,  8.7189e-02, -2.9874e-01,  2.7464e-01, -3.4620e-02,
        -1.8526e-01, -4.0381e-01, -3.2950e-01, -3.1077e-01, -9.6205e-01,
         1.8459e-01,  8.9893e-01,  1.0140e-01, -2.2178e-03, -1.2092e+00,
         4.6842e-01,  7.2310e-01,  2.6946e-01,  6.0602e-01,  3.3600e-01,
        -6.9873e-01,  4.4655e-01,  1.2239e+00, -1.9887e-01,  1.1207e+00,
         7.1256e-02, -9.7466e-02,  3.7977e-01, -4.9891e-01, -4.6798e-02,
         4.9862e-01,  4.8732e-01,  1.0820e-01, -1.8058e-01, -3.7724e-01,
        -2.1718e-01,  3.7783e-02,  4.1661e-01, -1.0268e-01, -8.5980e-01,
         2.2138e-01, -2.2971e-01,  6.2703e-02, -1.4382e-01,  1.4447e-01,
        -1.2565e-01, -1.5106e+00,  2.0652e-01,  7.5550e-01, -2.5890e-01,
        -6.2316e-01, -6.9404e-01, -1.0389e-01, -8.5084e-02, -7.4667e-02,
        -6.8613e-01,  1.2332e-01,  2.0314e-02, -3.4862e-01, -2.1305e-01,
         8.3751e-01,  1.4084e-01,  9.8267e-01, -1.2952e-01,  1.3081e-01,
         2.9403e-01, -3

In [6]:
encoded = bert_tokenizer.encode_plus(df['Text'][0], is_split_into_words=False, return_tensors="pt")

In [7]:
def get_token(i):
    token_ids_word = np.where(np.array(encoded.word_ids()) == i)
    word_piece = bert_tokenizer.convert_ids_to_tokens(encoded['input_ids'].squeeze()[token_ids_word])
    word_piece = [w.split("##")[1] if w.startswith("##") else w for w in word_piece]
    print(''.join(word_piece))

In [8]:
print(df['Text'][0][274])

h


In [9]:
test_string = "This is a test string."
indices = [i for i, x in enumerate(df['Text'][0]) if x == " "]
print([i for i  in indices if i > 274][0])
df['Text'][0][274:277]

277


'her'

In [10]:
def get_token_index_from_char_offset(char_offset, text):
    indices = [i for i, x in enumerate(text) if x == " "]
    end_idx = [i for i  in indices if i > char_offset][0]
    return text[char_offset:end_idx]
    
print(get_token_index_from_char_offset(274, df['Text'][0]))

her


In [11]:
tokens = [get_token(i) for i in range(90)]

Zoe
Telford
-
-
played
the
police
officer
girlfriend
of
Simon
,
Maggie
.
Dumped
by
Simon
in
the
final
episode
of
series
1
,
after
he
slept
with
Jenny
,
and
is
not
seen
again
.
Phoebe
Thomas
played
Cheryl
Cassidy
,
Pauline
'
s
friend
and
also
a
year
11
pupil
in
Simon
'
s
class
.
Dumped
her
boyfriend
following
Simon
'
s
advice
after
he
wouldn
'
t
have
sex
with
her
but
later
realised
this
was
due
to
him
catching
crabs
off
her
friend
Pauline


In [12]:
count = 0
indices = []
for i in range(df.shape[0]):
    is_okay = df['Text'][i].index(df['A'][i]) == df['A-offset'][i] and df['Text'][i].index(df['B'][i]) == df['B-offset'][i] \
            and df['Text'][i].index(df['Pronoun'][i]) == df['Pronoun-offset'][i]
    if is_okay:
        indices.append(i)
        count += 1

In [13]:
len(indices)

474

In [14]:
filtered_df = df[df.index.isin(indices)]

In [15]:
filtered_df = filtered_df.reset_index()

In [16]:
filtered_df

,index,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
1,7,development-8,Slant Magazine's Sal Cinquemani viewed the alb...,his,337,Greg Kot,173,False,Robert Christgau,377,True,http://en.wikipedia.org/wiki/The_Truth_About_L...
2,9,development-10,Shaftesbury's UK partners in the production of...,she,329,Christina Jennings,196,True,Kirstine Stewart,226,False,http://en.wikipedia.org/wiki/Murdoch_Mysteries
3,10,development-11,William Shatner portraying writer Mark Twain; ...,his,300,Peter Mansbridge,168,False,David Onley,212,True,http://en.wikipedia.org/wiki/Murdoch_Mysteries
4,28,development-29,"Meet Mike, the shortest bully to appear on the...",him,250,Mayhem Miller,153,False,Eddie Alvarez,189,False,http://en.wikipedia.org/wiki/List_of_Bully_Bea...
...,...,...,...,...,...,...,...,...,...,...,...,...
469,1987,development-1988,"On the morning of her disappearance, Alison st...",her,18,Alison,37,True,Jenna,101,False,http://en.wikipedia.org/wiki/Alison_DiLaurentis
470,1993,development-1994,"She then determined on becoming an artist, and...",her,407,Louise Breslau,363,True,Bashkirtseff,384,False,http://en.wikipedia.org/wiki/Marie_Bashkirtseff
471,1994,development-1995,Bill Grainer Bill Grainer is a Grammy certifie...,her,212,Linda Eder,131,False,Jennifer Hudson,147,True,http://en.wikipedia.org/wiki/Bill_Grainer
472,1997,development-1998,Grant played the part in Trevor Nunn's movie a...,she,348,Maria,259,True,Imelda Staunton,266,False,http://en.wikipedia.org/wiki/Sir_Andrew_Aguecheek


In [17]:
filtered_df['Text'].index(filtered_df['Pronoun'])

TypeError: 'RangeIndex' object is not callable

In [ ]:
idxs = []
for i in range(filtered_df.shape[0]):
    idx = filtered_df['Text'][i].index(filtered_df['Pronoun'][i])
    idxs.append(idx)

In [ ]:
idxs

In [ ]:
def get_word_embedding(encoded, output, idx):
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    print(token_ids_word)
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0)

encoded = bert_tokenizer.encode_plus(filtered_df['Text'][0], is_split_into_words=False, return_tensors="pt")
encoded_dict = {**encoded}
input_ids = encoded_dict['input_ids']
token_type_ids = encoded_dict['token_type_ids']
attention_mask = encoded_dict['attention_mask']

n = input_ids.shape[1]

passes = [(i * 256, i * 256 + 512) for i in range(int(n / 256) - 1)]
if len(passes) == 0:
    # Documents < 512 tokens
    with torch.no_grad():
        outputs = bert_model(**encoded)

    # Get all hidden states
    states = outputs.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i].squeeze() for i in [-4, -3, -2, -1]])
    query_t = torch.mean(output, dim=0)
    print(get_word_embedding(encoded, query_t, 90))

In [ ]:
encoded = bert_tokenizer.encode_plus(filtered_df['Text'][0], is_split_into_words=False, return_tensors="pt")
encoded_dict = {**encoded}
input_ids = encoded_dict['input_ids']
token_type_ids = encoded_dict['token_type_ids']
attention_mask = encoded_dict['attention_mask']

bert_tokenizer.convert_ids_to_tokens(input_ids)

In [ ]:
tokens = bert_tokenizer.convert_ids_to_tokens(input_ids.squeeze())
tokens.index(filtered_df['Pronoun'][0])
tokens.index(filtered_df['A'][0])
tokens.index(filtered_df['B'][0])

In [ ]:
count = 0
indices = []
for i in range(df.shape[0]):
    is_okay = df['Text'][i].index(df['A'][i]) == df['A-offset'][i] and df['Text'][i].index(df['B'][i]) == df['B-offset'][i] \
            and df['Text'][i].index(df['Pronoun'][i]) == df['Pronoun-offset'][i]
    if is_okay:
        indices.append(i)
        count += 1

In [ ]:
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
spacy_tokenizer = nlp.tokenizer

tokens = spacy_tokenizer(df['Text'][0])

In [ ]:
[token.text for token in tokens]

In [ ]:
len([token for token in tokens])

In [ ]:
len(df['Text'][0].split(" "))

In [ ]:
bert_tokenizer.encode_plus([token.text for token in tokens], return_tensors='pt', is_split_into_words=True)

In [23]:
def bs(list_, target_):
    lo, hi = 0, len(list_) -1

    while lo < hi:
        mid = lo + int((hi - lo) / 2)

        if target_ < list_[mid]:
            hi = mid
        elif target_ > list_[mid]:
            lo = mid + 1
        else:
            return mid + 1
    return lo

def bs_(list_, target_):
    lo, hi = 0, len(list_) -1

    while lo < hi:
        mid = lo + int((hi - lo) / 2)

        if target_ < list_[mid]:
            hi = mid
        elif target_ > list_[mid]:
            lo = mid + 1
        else:
            return mid
    return lo

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(df['Text'][0])


# char offset to token offset
lens = [token.idx for token in doc]
mention_offset1 = bs(lens, 274) - 1
# mention_offset2 = bs(lens, char_offset2) - 1

In [ ]:
mention_offset1

In [ ]:
doc[57]

In [20]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [24]:
pronoun_token_offsets, a_token_offsets, b_token_offsets, text_tokens = [], [], [], []

for i in range(df.shape[0]):
    pronoun_offset = df['Pronoun-offset'][i]
    a_offset = df['A-offset'][i]
    b_offset = df['B-offset'][i]
    doc = nlp(df['Text'][i])
    lens = [token.idx for token in doc]
    pronoun_token_offsets.append(bs(lens, pronoun_offset) - 1)
    a_token_offsets.append(bs(lens, a_offset) - 1)
    b_token_offsets.append(bs(lens, b_offset) - 1)
    text_tokens.append([token.text for token in doc])

token_df = pd.DataFrame(list(zip(pronoun_token_offsets, a_token_offsets, b_token_offsets, text_tokens)),
                  columns=["Pronoun-token-offset", "A-token-offset", "B-token-offset", "Text Tokens"])

In [25]:
token_df

,Pronoun-token-offset,A-token-offset,B-token-offset,Text Tokens
0,57,39,42,"[Zoe, Telford, --, played, the, police, office..."
1,56,46,49,"[He, grew, up, in, Evanston, ,, Illinois, the,..."
2,54,35,49,"[He, had, been, reelected, to, Congress, ,, bu..."
3,66,35,70,"[The, current, members, of, Crime, have, also,..."
4,85,42,57,"[Her, Santa, Fe, Opera, debut, in, 2005, was, ..."
...,...,...,...,...
1995,84,53,63,"[Faye, 's, third, husband, ,, Paul, Resnick, ,..."
1996,50,22,45,"[The, plot, of, the, film, focuses, on, the, l..."
1997,72,56,58,"[Grant, played, the, part, in, Trevor, Nunn, '..."
1998,56,27,42,"[The, fashion, house, specialised, in, hand, -..."


In [26]:
df = pd.concat([df,token_df],axis=1)

In [ ]:
df['Text'][0][df['A-token-offset'][0]]

In [ ]:
tokenizer = nlp.tokenizer

In [ ]:
len([token for token in tokenizer(df['A'][0])])

In [ ]:
[token.text for token in doc][df['A-token-offset'][0]:df['A-token-offset'][0]+len([token for token in tokenizer(df['A'][0])])]

In [ ]:
class Document:
    def __init__(self, tokens, corefs, speakers, sentences, filename):
        self.tokens = tokens
        self.corefs = corefs
        self.speakers = speakers
        self.sentences = sentences
        self.filename = filename

        self.tags = None

In [ ]:
class GapDocument:
    def __init__(self, tokens, corefs, filename):
        self.tokens = tokens
        self.corefs = corefs
        self.filename = filename

        self.tags = None

    def __getitem__(self, idx):
        return (self.tokens[idx], self.corefs[idx])

    def __repr__(self) -> str:
        return f'Document containing {len(self.tokens)} tokens'

    def __len__(self):
        return len(self.tokens)

    @cached_property
    def sents(self):
        """ Regroup raw_text into sentences """

        # Get sentence boundaries
        sent_idx = [idx+1
                    for idx, token in enumerate(self.tokens)
                    if token in ['.', '?', '!']]

        # Regroup (returns list of lists)
        return [self.tokens[i1:i2] for i1, i2 in bert_pairwise([0] + sent_idx)]

    def spans(self):
        return [Span(i1=i[0], i2=i[-1], id=idx,
                     speaker=None)
                for idx, i in enumerate(compute_idx_spans(self.sents))]

In [ ]:
def load_gap_file(filename):
    df = pd.read_csv(filename, sep='\t')

    documents = []

    for i in range(df.shape[0]):
        # Tokenize GAP example
        doc = nlp(df['Text'][i])

        # Convert char offset to token offset
        lens = [token.idx for token in doc]
        pronoun_offset = bs(lens, df['Pronoun-offset'][i]) - 1
        a_offset = bs(lens, df['A-offset'][i]) - 1
        b_offset = bs(lens, df['B-offset'][i]) - 1

        doc = GapDocument([token.text for token in doc], {'pronoun': pronoun_offset, }, filename)
        documents.append(doc)

    #
    # token_df = pd.DataFrame(list(zip(pronoun_token_offsets, a_token_offsets, b_token_offsets, text_tokens)),
    #                         columns=["Pronoun-token-offset", "A-token-offset", "B-token-offset", "Text Tokens"])
    #
    # df = pd.concat([df, token_df], axis=1)

    return documents
